# Создание цепочки извлечения данных

Раздел содержит руководствопо разработке цепочки для извлечения структурированной информации из неструктурированного текста.

:::important

Примеры в разделе работают только с моделями, которые поддерживают вызов функций/инструментов.

:::

## В этом руководстве

Примеры в разделе показывают как:
- использовать [чат-модели](/docs/concepts/#chat-models)
- вызывать [функции/инструменты](/docs/concepts/#function-tool-calling)
<!--
- Отладка и трассировка вашего приложения с помощью [LangSmith](/docs/concepts/#langsmith)
-->

## Подготовка

### Jupyter

Как и многие другие руководства в документации GigaChain это руководство основано на [блокноте Jupyter](https://jupyter.org/).
Блокноты предоставляют интерактивность, которая значительно упрощает изучение работы с LLM.

Об установке Jupyter читайте в [официальной документации](https://jupyter.org/install).

### Установка Зависимостей

В примере используются модели GigaChat (чат-модель и эмбеддинги), а также векторное хранилище Chroma, но в своем приложении вы можете использовать любой из доступных компонентов.

Для работы с примером нужно установить пакеты:

```python
%pip install --upgrade --quiet  gigachain langchainhub chromadb bs4
```


For more details, see our [Installation guide](/docs/how_to/installation).

<!--
### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls.
As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent.
The best way to do this is with [LangSmith](https://smith.langchain.com).

After you sign up at the link above, make sure to set your environment variables to start logging traces:

```shell
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_API_KEY="..."
```

Or, if in a notebook, you can set them with:

```python
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

## Определение схемы данных

В первую очередь вам нужно описать, какую информацию вы хотите извлечь из текста.

В примере ниже с помощью Pydantic-модели задана схема образца персональных данных, которые нужно извлечь с помощью модели.

In [5]:
from typing import Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Док-строка для сущности Person.
    # Эта док-строка передается в модель в качестве описания схемы Person.
    # Она может помочь улучшить результаты извлечения.

    # Обратите внимание:
    # 1. Каждое поле является необязательным (`optional`). Это позволяет модели отказаться от его извлечения.
    # 2. Каждое поле имеет описание (`description`). Это описание использует модель.
    # Хорошее описание может помочь улучшить результаты извлечения данных.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )

При определении схемы придерживайтесь следующих рекомендаций:

- Документируйте как атрибуты, так и саму схему. Эта информация передается в модель и используется для повышения качества извлечения данных.
- Не вынуждайте модель придумывать данные. В приведенном выше примере все атрибуты необязательны (параметр `Optional`). Таким образом модель сможет вернуть `None`, если в тексте нет подходящих для извлечения данных.

## Экстрактор

Создайте экстрактор информации, используя заданную выше схему данных.


In [6]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Определите собственный промпта, чтобы предоставить модели инструкции и другой дополнительный контекст.
# 1) Вы можете повысить качесво извлечения, если укажете в шаблоне промпа образцы ожидаемых результатов
# 2) Задайте дополнительные параметры, чтобы учитывать контекст
# (например, добавьте метаданные о документе, из которого был извлечен текст.)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # О том как использовать примеры для повышения качетсва результатов
        # вы можете узнать в соответсвующем руководстве.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

Для запуска примера используйте модель, которая поддерживает вызов функций/инструментов.

Вам подойдет любая модель GigaChat, предназначенная для генерации.

In [7]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Person)

/Users/harrisonchase/workplace/langchain/libs/core/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The method `ChatMistralAI.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Проверьте работоспособность:

In [8]:
text = "Alan Smith is 6 feet tall and has blond hair."
runnable.invoke({"text": text})

Person(name='Alan Smith', hair_color='blond', height_in_meters='1.83')

<!--
:::important

Извлечение является генеративным 🤯

LLMs являются генеративными моделями, поэтому они могут делать некоторые довольно крутые вещи, например, правильно извлекать рост человека в метрах, даже если он был предоставлен в футах!
:::

Мы можем увидеть трассировку LangSmith здесь: https://smith.langchain.com/public/44b69a63-3b3b-47b8-8a6d-61b46533f015/r
-->

## Извлечение нескольких сущностей

Как правило вам будет нужно извлекать список сущностей, а не одну сущность.

Для этого вы можете вкалдывать Pydantic-модели друг в друга.

In [9]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Док-строка для сущности Person.
    # Эта док-строка передается в модель в качестве описания схемы Person.
    # Она может помочь улучшить результаты извлечения.

    # Обратите внимание:
    # 1. Каждое поле является необязательным (`optional`). Это позволяет модели отказаться от его извлечения.
    # 2. Каждое поле имеет описание (`description`). Это описание использует модель.
    # Хорошее описание может помочь улучшить результаты извлечения данных.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Создает схему, чтобы мы могли извлекать несколько сущностей.
    people: List[Person]

<!--
:::{.callout-important}
Извлечение здесь может быть не идеальным. Пожалуйста, продолжайте читать, чтобы узнать, как использовать **Референсные примеры** для улучшения качества извлечения, и смотрите раздел **Руководства**!
:::
-->

In [10]:
runnable = prompt | llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
runnable.invoke({"text": text})

Data(people=[Person(name='Jeff', hair_color=None, height_in_meters=None), Person(name='Anna', hair_color=None, height_in_meters=None)])

:::tip

Схема, которая поддерживает извлечение нескольких сущностей, также позволяет модели не извлекать ни одной сущности, если в тексте нет соответствующей информации. В таком случае модель вернет список.

Как правило это хорошо, так как позволяет указывать обязательные атрибуты для сущности без необходимости вынуждать модель обнаруживать эту сущность.
:::

<!--
Трассировку LangSmith можно увидеть здесь: https://smith.langchain.com/public/7173764d-5e76-45fe-8496-84460bd9cdef/r
-->

## Смотрите также

- [Добавление образцов извлеченных данных](/docs/how_to/extraction_examples)
- [Обработка длинных текстов](/docs/how_to/extraction_long_text)
- [Извлечение данных на основе парсинга](/docs/how_to/extraction_parse)